In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

### Importing the libraries

In [ ]:
# pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
HORIZONTAL = "horizontal"
VERTICAL = "vertical"
HORIZONTAL_AND_VERTICAL = "horizontal_and_vertical"
tf.keras.layers.RandomFlip(
    mode=HORIZONTAL_AND_VERTICAL, seed=None
)

<RandomFlip name=random_flip_4, built=False>

In [ ]:
height_factor=(0.2, 0.3)
tf.keras.layers.RandomZoom(
    height_factor,
    width_factor=None,
    fill_mode='reflect',
    interpolation='bilinear',
    seed=None,
    fill_value=0.0,
    data_format=None
)

<RandomZoom name=random_zoom_4, built=False>

In [ ]:
scale=1./255
tf.keras.layers.Rescaling(
    scale, offset=0.0
)

<Rescaling name=rescaling_6, built=False>

In [ ]:
IMG_SIZE = 180
HORIZONTAL = "horizontal"
VERTICAL = "vertical"
height_factor=(0.2, 0.3)

train_datagen = tf.keras.Sequential(
  [
    # layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1./255),
    layers.RandomFlip(mode=HORIZONTAL_AND_VERTICAL, seed=None),
    layers.RandomZoom(
      height_factor,
      width_factor=None,
      fill_mode='reflect',
      interpolation='bilinear',
      seed=None,
      fill_value=0.0,
      data_format=None
    )
  ]
)

### Preprocessing the Test set

In [ ]:
test_datagen = tf.keras.Sequential([
    # layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1./255)
])

### Upload in Colab

In [ ]:
# from google.colab import files
# files.upload()

### Extract the contents

In [ ]:
# import tarfile

# # Extract to current directory
# with tarfile.open("cifar-100-python.tar.gz", "r:gz") as tar:
#     tar.extractall()

### Load and preprocess CIFAR-100 data

In [ ]:
train_dict = unpickle('cifar-100-python/train')
test_dict = unpickle('cifar-100-python/test')

<pre>
Reshape flat CIFAR-100 image data into (num_samples, channels, height, width)
Each image has 3072 values: 1024 each for R, G, B channels
Using -1 lets NumPy automatically infer the number of samples (should be 50000)
Transpose to (num_samples, height, width, channels) to match TensorFlow's expected format
Convert fine-grained class labels (0–99) from a Python list to a TensorFlow tensor
This is needed for compatibility with tf.data and model training
</pre>

### Reshape and reorder axes

In [ ]:
X_test = test_dict[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1).astype('float32')
y_test = tf.convert_to_tensor(test_dict[b'fine_labels'])

In [ ]:
X_train = train_dict[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1).astype('float32')
y_train = tf.convert_to_tensor(train_dict[b'fine_labels'])

### Build tf.data.Dataset and apply train_datagen & test_datagen pipelines

In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
# Create a tf.data.Dataset from training images and labels
# Each element will be a (image, label) pair
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# Shuffle the dataset to randomize the order of samples
# A buffer size of 10000 provides good mixing
train_ds = train_ds.shuffle(10000)
# Group the dataset into batches of size BATCH_SIZE
train_ds = train_ds.batch(BATCH_SIZE)
# Apply the preprocessing and data augmentation defined in train_datagen
# Resizing, rescaling, flipping, and zooming will be applied on-the-fly
# train_ds = train_ds.map(lambda x, y: (train_datagen(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (x, y), num_parallel_calls=AUTOTUNE)

# Prefetch batches to improve training performance by overlapping data loading and model
train_ds = train_ds.prefetch(AUTOTUNE)

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.batch(BATCH_SIZE)
# test_ds = test_ds.map(lambda x, y: (test_datagen(x), y), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (x, y), num_parallel_calls=AUTOTUNE)

test_ds = test_ds.prefetch(AUTOTUNE)

### Part 2 - Building the CNN

## Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(32, 32, 3), padding='same', strides=(2, 2)))
cnn.add(tf.keras.layers.BatchNormalization())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.Dropout(0.25)) # Dropout after pooling

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same', strides=(2, 2)))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='same'))
cnn.add(tf.keras.layers.Dropout(0.25))

### Adding a third convolutional layer

In [ ]:
# cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
# cnn.add(tf.keras.layers.BatchNormalization())
# cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# cnn.add(tf.keras.layers.Dropout(0.25))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.Dropout(0.3)) # More aggressive dropout before final layer

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=100, activation='softmax'))

### Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
from tensorflow.keras.optimizers import Adam
loss = 'sparse_categorical_crossentropy', # because labels are integer class
cnn.compile(
    optimizer = Adam(learning_rate=2.5000e-04),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
  )

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(
monitor='val_loss', # What metric to watch
factor=0.5, # Reduce LR by half
patience=2, # Wait 3 bad epochs before acting
verbose=1 # Print when LR changes
)
cnn.fit(train_ds,
validation_data = test_ds,
verbose=1,
callbacks=[lr_scheduler],
epochs = 25)

Epoch 1/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.0299 - loss: 4.8041 - val_accuracy: 0.1219 - val_loss: 3.8929 - learning_rate: 2.5000e-04
Epoch 2/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.0844 - loss: 4.0586 - val_accuracy: 0.1678 - val_loss: 3.5981 - learning_rate: 2.5000e-04
Epoch 3/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.1216 - loss: 3.7856 - val_accuracy: 0.1991 - val_loss: 3.4471 - learning_rate: 2.5000e-04
Epoch 4/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.1480 - loss: 3.6079 - val_accuracy: 0.2157 - val_loss: 3.3109 - learning_rate: 2.5000e-04
Epoch 5/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 43s 17ms/step - accuracy: 0.1738 - loss: 3.4721 - val_accuracy: 0.2249 - val_loss: 3.2578 - learning_rate: 2.5000e-04
Epoch 6/25
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 27s 17ms/step - accuracy: 0.1892 - loss: 3.3654 - val_accuracy: 0.2488 - val_loss: 3.1358 - learning_rate: 2.5000e-04
Epoch 7/25
1563/1563 ━━━━━━━━━━━━━━━━━━━

### Finally, evaluate its performance

In [ ]:
# Evaluate the trained model on the test dataset
test_loss, test_accuracy = cnn.evaluate(test_ds)
# Print the results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3623 - loss: 2.5172
Test Loss: 2.5255
Test Accuracy: 0.3611
